In [42]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Global TF-IDF Vectorizer to avoid redundant instantiation
vectorizer = TfidfVectorizer()

In [43]:
# Sample shapes and edges for graph 1
shapes1 = [
    {'id': 1, 'type': 'terminal', 'center': (0, 0), 'text': 'Hello World'},
    {'id': 2, 'type': 'arrow-down', 'center': (1, 1), 'text': 'Graph Comparison'},
    {'id': 3, 'type': 'terminal', 'center': (2, 2), 'text': 'Python Code'},
]

edges1 = [
    {'source': 1, 'target': 2},
    {'source': 2, 'target': 3},
]

# Sample shapes and edges for graph 2
shapes2 = [
    {'id': 1, 'type': 'terminal', 'center': (0, 0), 'text': 'Hello World!'},
    {'id': 2, 'type': 'arrow-down', 'center': (1, 1), 'text': 'Graph Similarity'},
    {'id': 3, 'type': 'terminal', 'center': (2, 2), 'text': 'Python Scripting'},
]

edges2 = [
    {'source': 1, 'target': 2},
    {'source': 2, 'target': 3},
    {'source': 3, 'target': 1},  # Extra edge in graph 2
]

In [44]:
# Step 1: Graph Construction
def build_graph(shapes, edges):
    """
    Build a directed graph from shapes and edges.

    Args:
        shapes (list): List of dictionaries containing shape attributes.
        edges (list): List of dictionaries containing edge connections.

    Returns:
        nx.DiGraph: A directed graph representing the flowchart.
    """
    G = nx.DiGraph()
    for shape in shapes:
        G.add_node(shape['id'], type=shape['type'], center=shape['center'], text=shape['text'])
    for edge in edges:  # edges based on arrows
        G.add_edge(edge['source'], edge['target'])
    return G

In [45]:
# Step 2: Text Similarity
def text_similarity(text1, text2):
    """
    Compute cosine similarity between two texts using TF-IDF.

    Args:
        text1 (str): First text.
        text2 (str): Second text.

    Returns:
        float: Cosine similarity between the two texts.
    """
    tfidf = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

In [46]:
# Step 3: Node Similarity
def node_similarity(node1, node2):
    """
    Compute similarity between two nodes based on their attributes.

    Args:
        node1 (dict): Attributes of the first node.
        node2 (dict): Attributes of the second node.

    Returns:
        float: Similarity score between the two nodes.
    """
    # Compare shape type (exact match)
    type_sim = 1 if node1['type'] == node2['type'] else 0

    # Compare text (cosine similarity using TF-IDF)
    text_sim = text_similarity(node1['text'], node2['text'])

    # Compare spatial position (Euclidean distance between centers)
    center1 = node1['center']
    center2 = node2['center']
    spatial_dist = ((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2) ** 0.5
    spatial_sim = 1 / (1 + spatial_dist)  # Normalize to [0, 1]

    # Weighted combination of similarities
    return 0.4 * type_sim + 0.4 * text_sim + 0.2 * spatial_sim

In [47]:
#Step 4: Edge Similarity
def edge_similarity(graph1, graph2):
    """
    Compute similarity between edges of two graphs.

    Args:
        graph1 (nx.DiGraph): First graph.
        graph2 (nx.DiGraph): Second graph.

    Returns:
        float: Edge similarity score.
    """
    common_edges = len(set(graph1.edges) & set(graph2.edges))
    total_edges = max(len(graph1.edges), len(graph2.edges))
    return common_edges / total_edges if total_edges > 0 else 0

In [48]:
#Step 5: Graph Similarity
def graph_similarity(graph1, graph2):
    """
    Compute structural similarity between two graphs.

    Args:
        graph1 (nx.DiGraph): First graph.
        graph2 (nx.DiGraph): Second graph.

    Returns:
        float: Structural similarity score.
    """
    # Node similarity
    node_sim = 0
    matched_nodes = 0
    for node1 in graph1.nodes:
        for node2 in graph2.nodes:
            sim = node_similarity(graph1.nodes[node1], graph2.nodes[node2])
            if sim > 0.7:  # Threshold for matching nodes
                node_sim += sim
                matched_nodes += 1
    node_sim = node_sim / matched_nodes if matched_nodes > 0 else 0

    # Edge similarity
    edge_sim = edge_similarity(graph1, graph2)

    # Weighted combination of node and edge similarity
    return 0.6 * node_sim + 0.4 * edge_sim

In [49]:
#Step 6: Overall similarity
def overall_similarity(graph1, graph2):
    """
    Compute overall similarity between two graphs, combining structural and textual similarity.

    Args:
        graph1 (nx.DiGraph): First graph.
        graph2 (nx.DiGraph): Second graph.

    Returns:
        float: Overall similarity score.
    """
    # Step 1: Structural similarity (already includes textual similarity in node_similarity)
    structural_sim = graph_similarity(graph1, graph2)

    # Step 2: Combine structural similarity with any additional metrics if needed
    # (In this case, no additional textual similarity is required)
    return structural_sim

In [50]:
# Main Workflow
graph1 = build_graph(shapes1, edges1)
graph2 = build_graph(shapes2, edges2)

# Calculate similarity score
similarity_score = overall_similarity(graph1, graph2)
print("Similarity Score:", similarity_score)

Similarity Score: 0.760442175030868
